In this notebook we try to estimate the dimensionality of the dynamic trajectory subspace.

### Helpful articles
- [Supplementary material from AI Poincare](https://journals.aps.org/prl/supplemental/10.1103/PhysRevLett.126.180604/poincare_supplemental_materialv2.pdf)
- [Interpretable conservation law estimation by deriving the symmetries of dynamics from trained deep neural networks](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.103.033303) cited in AI Poincare.

In [ ]:
from experiments import *
import autoencoders as ae

In [ ]:
import wandb

In [ ]:
sweep_configuration = {
    'method': 'random',
    'name': 'sweep kl1',
    'metric': {
        'goal': 'minimize',
        'name': 'coupled_oscillator_2_test_mse'
    },
    'parameters': {
        'k': {'values': [1, 3, 5, 10]},
        'l1': {'values': [0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-1, 1]},
    }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project="AE new2 (co sweep)")

In [ ]:
def main():
    wandb.init(project="AE new2 (co sweep)")
    k = wandb.config.k
    l1 = wandb.config.l1
    for exp in [CoupledOscillator]:
        suite = ae.TrajectoryAutoencoderSuite(exp, criterion=ae.GAELoss(k=k), additional_loss=ae.sparse_and_l1(l1_lambda=l1), epochs=2000, do_animate=True, log_prefix=f"{k}_{l1}")
        suite.analyze_n_eff(42)

In [ ]:
wandb.agent(sweep_id, function=main)

In [ ]:
wandb.finish()